# Урок 3. Коллаборативная фильтрация

В качестве домашнего задания ответьте на следующие вопросы:

### 1. В чем принципиальные отличия item-item рекомендаций от ALS?

Метод item-item рекомендациий работает через оптимизацию напрямую в матрице пользователь – товар, которая в реальных задачах получается огромной, очень разреженной и зашумленной.

ALS (alternating least squares) предварительно делает разложение исходной матрицы предпочтений на произведение двух матриц факторов пользователей и факторов товаров, которые на порядок меньше, менее разряжены и зашумлены, и затем работает по принципу минимизации среднеквадратичной ошибки на проставленных рейтингах, причем оптимизация происходит поочередно, сначала по факторам пользователей, потом по факторам товаров.

### 2. Приведите 3 примера весов user-item матрицы для задачи рекоммендаций товаров

### 3. Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)